Problem Set 1.2: Introduction to Python with dimensionality reduction and classification.
---

This is a Jupyter Notebook that executes commands in the Python programming language. Just like the R Markdown Notebook, when you execute the code in a cell the result will appear beneath the code. For a quick introduction to Python, see [here](https://cs231n.github.io/python-numpy-tutorial/).

You can execute cells by placing your cursor inside a cell and clicking the *Run* button (play button) at the top of the notebook window, just beneath the tab bar or by pressing *Ctrl/Cmd++Enter*.

You can add new chunk by clicking the *Insert Cell* (plus sign) button on the toolbar.

Make sure you are using the **Assignment 1** kernel. This will (hopefully) ensure you have the necessary packages to run everything in the notebook. You can change the kernel by clicking on the button showing the active kernel at the top-right of the notebook or with the "Kernel" drop down menu on the menu bar.

If the bash kernel does not appear as an option, open a new terminal (on the menu bar click *File* > *New* > *Terminal*) and type:

`python -m bash_kernel.install`

This should add the bash kernel as an option to Jupyter permanently.

**When you are finished, save and download the notebook. You will need to submit it through Canvas once you have finished both sections.**

Start by running this cell which will import the necessary packages and functions.

In [ ]:
import numpy as np
from scipy.io import mmread
import sklearn.decomposition as decomp
from sklearn.preprocessing import scale
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import os
from pathlib import Path
import urllib

gcp_url = "https://storage.googleapis.com/mit-cmn-neurogen-course-data/Assignment1"
write_path = "data_assignment1"
Path(write_path).mkdir(parents=True, exist_ok=True)
pset_dat = ["p3_coldata_zQ_sc_single.tsv", "p3_counts_zQ_sc_single.mtx", "p3_rowdata_zQ_sc_single.tsv",
            "p5_counts_zQ_sc_classification.mtx", "p5_labels_zQ_sc_classification.tsv", "p5_rowdata_zQ_sc_classification.tsv"]
for f in pset_dat:
    urllib.request.urlretrieve("/".join([gcp_url, f]), os.path.join(write_path, f))

# Part 3: Dimensionality reduction with PCA

Before we begin, you should familoarize yourself with the `numpy` package, which is the main python package for math and matrix manipulation. Refer to the tutorial, quickstart quide, and documentation here:


1. [Numpy quickstart guide](https://numpy.org/doc/stable/user/quickstart.html)
2. [Numpy documentation](https://numpy.org/doc/stable/)

## Background
The are ~6.2 billion bases in the human genome, any subset of which can be a feature of interest. Transcriptomics deals with studying the 50,000+ transcribed genes encoded by these bases, and epigenomics looks at the many more regulatory elements. A GWAS study may consider hundreds of thousands to millions of candidate single nucleotide polyphormisms for disease or trait associations. The common theme among genomic studies is that they are almost always high-dimensional, that is to say there are many features to account for, often far more than we are able or even care to. In the interest of time, resources, and interpretability we often look for ways to reduce our search space to focus on features of interest and relevance, but we don't ncessarily want to discard information. After all, when there are literally a million things to look at, how can we know in advance which ones matter or even how to begin proritizing them? 

Dimensionality reduction is a nethod for accomplishing exactly that. In essence, dimensionality reduction is the process by which we transform high-dimensional data into a much more palletable low-dimensional representation with minimal loss of information. The last part is critical. We don't want to lose anything of importance while shedding 95% of our data, but you might be surprised to know that it's actually easier done than said. Let's illustrate this with an example. A typical car has about 30,000 parts of which ~25,000 belong to the category of screws, grommets and plastic clips, several thousand of which are there just to hold things neatly in place while the car is assembled and could be stripped without affecting anything or without anyone noticing. Similarly, every mammalian cell expresses polymerases, histones, and ubiquitins, and while these may be more important than the zip ties in your car, more often than not they tell us equally as much about the cell's identity, functional specialization, or susceptibility in disease. Except under rare circumstances, these are examples of features that we can condense, if not outright discard. In doing so we not only have less features to worry about, but we know that whatever is left over is more important.

There is a plethora of dimensionality reduction algorithms used in genomics, but we'll focus on one of most intuitive and practical ones called [*Principal Component Analysis (PCA)*](https://en.wikipedia.org/wiki/Principal_component_analysis). A thorough treatment of PCA is the subject of a linear algebra or machine learning course, but in brief, it is a method to transform a dataset of $n$ observations and $p>n$ features into a set of still $n$ observations but with at most $n-1$ new features called *components*. Every component vector is orthogonal to every other one and each explains some exclusive subset of the variance in your data. The components are sorted such that the first component explains the greatest amount of variance, the second component explains second most, and so on, such that all $n-1$ components explain 100% of the cross-sample variation present in your data. This reduced representation is not only more intuitive, it's more computationally tractable, especially when dealing with enormous datasets, and we can visualize the components to discover patterns that we would've otherwise missed.

I didn't have time to put together some fancy figures illustrating the geometric intuition behind PCA, but when I Googled "pca for dummies" [I found someone who did](http://www.billconnelly.net/?p=697), although if you'd like a full blown tutorial there's [this](https://ourarchive.otago.ac.nz/bitstream/handle/10523/7534/OUCS-2002-12.pdf?sequence=1&isAllowed=y) as well.

## Computing the principal components

In this section, we will apply PCA to a small, but respectably high-dimensionsal single-cell RNA-seq dataset. This data represents the transcriptional profiles of three cell types found in the striatum of a 6 month old mouse. We will use PCA to convert our minimally-processed gene count data into a much more compact form and use that reduced representation to identify the cell types present as well as which genes are responsible for driving their variation.

Let's load the count matrix and cell metadata. 

In [ ]:
cts = mmread("data_assignment1/p3_counts_zQ_sc_single.mtx")
cts = cts.toarray()

Our count matrix is in the same format as in the previous section, where columns represent samples, and rows are genes. The only difference here is that the data was stored in sparse format. See example below for sparse vs dense data representations. 

This is because each sample corresponds to just one cell and these types of data ets usually range from thousands to millions of cells. However, since this particular dataset was sub-sampled to just a few hundred cells, we took the liberty of densifying it into a numpy array. We will deal extensively with sparse data structures in this course, but for now we will work exclusively with dense matrices since it will simplify things considerably.

Let's practice doing simple mathematical operations on numpy arrays. We imported the `numpy` package as `np`, meaning that you can (and must) use this abbreviation to call functions belonging to the package (e.g. `np.mean()`). Column-normalize the counts just like you did before using the applicable numpy functions. For reference, here is the normalization formula again.
$$ \large
\mathbf{\bar x_{j}} = M \frac{\mathbf{x_j}}{\sum{\mathbf{x_j}} }, \quad j=1,...,n 
$$
with
$$ \large
M=\text{median} \bigg (\sum{\mathbf{x_1}}, ..., \sum{\mathbf{x_n}} \bigg ) \text.
$$
Assign the normalized array to `cts_norm`.

In [ ]:
cts_norm = ... # Fill this in

Now something important to note is that normalization is not sufficient for the algorithm to work. For PCA to work properly, our features need to be centered and scaled to unit vectors. By convention, the matrix to be decomposed has observations as rows and features as columns, so we will first transpose our normalized counts and then center and scale the columns (now features). Th eresultant matrix to be passed to the algorithm is computed as

$$ \large
    \bar{X}^T = \frac{X^T - \mathbf{\mu}_{\text{feat}} }{\mathbf{\sigma_{\text{feat}}}}\text{.}
$$

This transforms our features to have a mean of 0 and standard deviation of 1. This type of matrix scaling (both of rows and columns) is common and a necessary preprocessing step in many contexts. You may not see it, but many tools you will use througout the course will perform this transformation under-the-hood before operating on the data. In the cell below implement this transformation and set the now transposed, centered, and scaled matrix to `Xs`.

In [ ]:
Xs = ... # Fill this in

We could have also centered the matrix using the `scale` function from the `sklearn.preprocessing` module as follows:

In [ ]:
from sklearn.preprocessing import scale
Xt = cts_norm.transpose()
Xs2 = scale(Xt, with_mean=True, with_std=True)

Let's verify whether the two transformations are equivalent 

In [ ]:
np.allclose(Xs, Xs2)

The output of this will be almost (but not exactly) identical to `Xs`. You can consult the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html#sklearn.preprocessing.scale) for `scale` to learn why that is. However all down stream output will be, for the most part, indistinguishable.

We will use the sciki-learn package (`sklearn`). We imported the `sklearn.decomposition` module as `decomp`. This module contains the functions needed to perform PCA.  First initialize a `PCA` object from which we can call the fitting and transformation function to transform our preprocessed counts.

In [ ]:
pca = decomp.PCA()
Xd = pca.fit_transform(Xs2)

We now have our reduce representation `Xd`. We can check the size to see just how "reduced" our new representation is.

In [ ]:
print(Xt.shape)
print(Xd.shape)

We have consolidated our 3669 genes into 438 new features (components). 

>But wait, that's 1 component too many. Since we have only $n-1$ degrees of freedom, we should have $n-1=437$ components. So what exactly is the 438th component? Let's take a look. Recall that unlike R, Python is 0-indexed. So the 438th component is at index 437.

In [ ]:
Xd[:, 437][:10]

>Our computer and numpy both use 64-bit percision, which allows us 16 digits of accuracy past the decimal point. Recall that we should have $n-1$ *non-zero* principal components, anything beyond would have to be a zero vector. As far as the computer is concerned, the 438th component is in fact all 0's.

We also have the `pca` object. Let's examine it's attributes. 


In [ ]:
print(pca.__dict__.keys())

Most interestingly the instance now contains:
* `components_`: The variable loadings (sometimes called the *rotation matrix*)
* `explained_variance_`:  The variance explined by each component
* `explained_variance_ratio_`: The fraction of total variance explained by each component. 

Run the following cell, and you will see what percentage of the variance is explained by the first 2 components. Further, you can see the sum across all the components explains all of it. There is no loss of information.

In [ ]:
print(pca.explained_variance_ratio_[:2] * 100)
print(np.sum(pca.explained_variance_ratio_) * 100)

The first two components explain barely 9% of our variance. That doesn't seem like much.

## Finding patterns
Let's load the metadata corresponding to the cells in count matrix that we started with.

In [ ]:
mdata = pd.read_table("data_assignment1/p3_coldata_zQ_sc_single.tsv", sep="\t", header=0)

Now we'll plot the first two of the seemingly uninformative PC's we computed and label the points with the annotations in the meta data.

In [ ]:
sns.scatterplot(x = Xd[:, 0], y = Xd[:, 1], hue = mdata["SubType"])

So despite explaining only ~9% of the count data, the first two components are sufficient to partition the three cell populations present. This is actaully quite impressive. To appreciate why that is, let's provide some background on these cell types. This single-cell data consists of [oligodendrocytes](https://en.wikipedia.org/wiki/Oligodendrocyte) and [medium spiny neurons](https://en.wikipedia.org/wiki/Medium_spiny_neuron), also known as spiny projection neurons (SPN) as labeled here. Oligodendrocytes are a type of glial cell responsible for mylenating the axons of neurons enabling [action potentials](https://en.wikipedia.org/wiki/Action_potential) to travel much faster than they would otherwise. SPNs are a type of inhibitory neuron and the most abundant cell type in the striatum. There are two types of SPNs, distinguished by their expression of D1-type and D2-type dopamine receptors. The ones in this dataset are D1-expressing. Distinguishing between an inhibitory neuron and a glial cell should be fairly trivial and indeed it's where most of the variance lies. However, each type of SPN can be further subdivided in to striosomal SPNs and matrix SPNs. The striosomomal SPNs are localized into visible patches called [striosomes](https://en.wikipedia.org/wiki/Striosome) and are all believed to have a distinct functional role to those that reside in the "matrix" (the area outside the striosomes). Until very recently, it was nearly impossible to distinguish striosomal from matrix SPNs by anything other than their spatial localization, and there was even debate about whether they differed at all. Nevertheless, using this data we can clearly partition the two populations along an axis that supposedly only accounts for 0.7% of our variance!

## Finding features
Before saying anything about using dimensionality reduction to find patterns in data, we mentioned using it to narrow down features of interest. Indeed PCA is foremost a [*decomposition*](https://en.wikipedia.org/wiki/Matrix_decomposition) method, meaning that it decomposes a matrix into [two or more matrices](http://www.statistics4u.com/fundstat_eng/cc_pca_loadscore.html). What each of those matrices encodes depends on the method, and there are many. For PCA, the input matrix is decomposed into a matrix of principal components which encodes the observations into a reduced feature space, and a matrix of variable loadings, often called the rotation. The rotation matrix encodes the features into a new space as well, but not a reduced space necessarily. However, the magnitude of the values in this matrix tells us how much of an effect that feature has on each principal component and the sign tells us if the feature is positively or negatively correlated with the component. We can also do what we did previously and visually inspect out loadings to find patterns among the features.

Load the metadata corresponding to the features of the count matrix.

In [ ]:
rdata = pd.read_table("data_assignment1/p3_rowdata_zQ_sc_single.tsv", sep="\t", header=0)
rdata.head()

Now let's visually inspect the first two loadings interactively using the `plotly` package. With this package you can generate a plot that allows you to zoom, pan, select, and see metadata about each individual point.

In [ ]:
import plotly.express as px
V = pca.components_.T # Extract and transpose the rotation matrix from the `pca` object. 
fig = px.scatter(rdata, x = V[:,0], y = V[:,1], color=rdata["marker"],hover_name=rdata["Gene"])
fig.show()

Using a different tool kit that we'll showcase later in the course, we identified the top 100 uniquely expressed marker genes for each of the cell types in this data and labeled them here accordingly. We can clearly see that by plotting the first two loadings, the genes in our data cluster by their selective expression in each of our cell types. The right corner of the cluster is heavily enriched for oligodendrocyte marker genes, many of which would be lowly expressed or entirely absent from neurons. As expected there is a much fuzzier boundary between the striosome and matrix SPN markers. The genes near the center line on the left which have no cell type association are either neuron-specific or SPN-specific genes that are not striosome or matrix enriched (e.g. *Nrg1*, *Nrxn3*, *Pde10a*, *Bcl11b*).

Last, part of decomposing a matrix implies that you can put it back together. Indeed we can multiply the PC's and the rotation matrix to recover the orignal (scaled) input matrix that we passed to the transformation function.

$$ \large
\bar{X} = X_{\text{PC}}V
$$


In [ ]:
np.allclose(Xs2, np.matmul(Xd, V.T))  # We need to un-invert V

We can also use the conventient `inverse_transform()` method in the `PCA` class to do this automatically.

In [ ]:
X_org = ... # Fill this in
np.allclose(Xs, X_org)

We can even get our counts back if we were to revert the scaling and add back the means.

## Discussion


PCA is one of many dimensionality reduction methods used in genomics. Other commonly used methods include non-negative matrix factorization (NMF) and linear discriminant analysis (LDA) which , like PCA, are examples of linear methods, but there are also some that transform data into reduced non-linear representations such as autoencoders, and the popular UMAP and t-SNE visualization algorithms. They all have their particular use cases and data contexts in which they can be applied, but ultimately they all represent ways of condensing data in ways that make it easier to identify and extract interesting characteristics of our data.  

# Part 4: Clustering and classification
We now move to the topics of deciding how to annotate, categorize, or otherwise desribe new data. When we conduct an experiment, especially one with a large number of samples or features, we are often given little information regarding where a data point came from, or what it represents. Even if we do, we are often left with the challenge of determining if and how the data points relate to one another. Dimensionality reduction and classification go hand in hand as one generally precedes the other. We may conduct an experiment with many variables, reduce and project our data to a low-dimensional space and behold... it's not random. There is structure, there are patterns, and we don't know what they are or why they're there. Since the second law of thermodynamics tells us that order does not generally occur in nature by chance (quite the opposite in fact) when we observe patterns in an experiment we are almost forced to assume that there is something interesting hiding in our data... or that we made a mistake. Either way, it's worth finding out what it is.

Imagine you just finished an experiment and recieved your data. You run your favorite dimensionality reduction algorithm to get a 2D representation of your data (that we'll assume is correct), and when you plot it you see this.

In [ ]:
R = data=np.random.rand(300,2)
c = np.random.choice([0,1,2], 300)
sns.scatterplot(x=R[:,0], y=R[:,1])

This looks like noise. You probably wouldn't be compelled to look much further.

However, if you plotted your data and it looked like the one in the cell below, it might pique your interest.

In [ ]:
from sklearn.datasets import make_blobs
R, c = make_blobs(n_samples=500, centers=3, n_features=2, random_state=0,  cluster_std=0.25)
sns.scatterplot(x=R[:,0], y=R[:,1])

But this is too easy. Let's consider the case where the data isn't as neat, but it's clear that there is some structure.

In [ ]:
from sklearn.datasets import make_blobs
R_km, c_km = make_blobs(n_samples=500, centers=4, n_features=2, random_state=0, cluster_std=0.85)
sns.scatterplot(x=R_km[:,0], y=R_km[:,1])

We will attempt to partition and categorize the points in the unlabeled dataset using the [*k*-means](https://en.wikipedia.org/wiki/K-means_clustering) algorithm. 
The naive *k*-means algorithm works as follows:
1. Randomly initalize *k* points the will serve as the centroids for classification.
2. Assign each point to the cluster corresponding to the nearest centroid.
3. Compute the new mean of the each cluster and make that the new centroid.
4. Repeat 2 and 3 until the centroid positions no longer change. 

Since *k* is an input to the algorithm, it requires us to know the the numper of clusters *a priori* in order to give an adequate result. For this example, you can determine *k* by visual inspection.

We're not going to manually implement *k*-means by hand. Although not at all difficult, it is time consuming and would not be of any educational value here. However, you will still be asked to do *k*-means clustering on this data. 
As part of the programming tutorial component of this assignemt, you will use the scikit-learn `KMeans()` implementation to classify the aforementioned dataset. `R_km` contains the *x* and *y* of the points you will be classifying. Refer to the [`KMeans()` documentation here](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) to implement the method. Assign the predicted labels to `c_test`.

In [ ]:
from sklearn.cluster import KMeans
c_pred = ... # Fill this in
sns.scatterplot(x=R_km[:,0], y=R_km[:,1], hue=c_pred) # Plot your labels

Now compare against the actual labels. They should look very similar.

In [ ]:
sns.scatterplot(x=R_km[:,0], y=R_km[:,1], hue=c_km)

*k*-means is an example of a *unsupervised* clustering algorithm. You may have heard the terms *supervised* and *unsupervised* thrown around in the context of machine learning. To put it in simple terms, an unsupervised algorithm operates on the input data exclusively and tries to learn the internal structure. Conversly, supervised algorithms learn by being *trained*, meaning they they are exposed to a labeled dataset that is used to "learn the pattern" so to speak and generate a model that can then be applied to make predictions about the labels of similar data.

There exists a somewhat (distantly) related supervised counterpart to *k*-means called the *k*-nearest neighbor (*k*-NN) classifier. *k*-NN is a classification (not clustering) algorithm. There is a difference. A clustering algorithm places elements into categories, whereas a classification algorithm ask which category the element belongs to. There are instances in which either type can be applied to same task, but more often than not this is due to the user failing to understand the distinction. For our purposes, which will be discussed shortly, the classifier is better suited for the job.

*k*-NN works by assigning each point in the test set the most common label among it's *k* neighbors in the training set. An interesting thing to note is that we don't even actually need a training set for *k*-NN if we have partially labeled data. In this case, the labeled points will effectively serve as the training set.

Let's train *k*-NN on a similar toy dataset and test it on a noisier version of the data. Let's define the data generation parameters. This data set will have just 2 features and be grouped into 5 clusters.

In [ ]:
n_samples = 500
centers = 5
seed = 0
n_features = 2

In [ ]:
R_train, c_train = make_blobs(n_samples=n_samples, centers=centers, n_features=n_features, random_state=seed,  cluster_std=0.7)
sns.scatterplot(x=R_train[:,0], y=R_train[:,1], hue = c_train)

Now let's generate the test set. We will use the same data as above as  a base, but will increase the cluster variance and add randomly distributed noise to every point.

In [ ]:
R_test, c_test = make_blobs(n_samples=n_samples, centers=centers, n_features=n_features, random_state=seed,  cluster_std=1.0)
noise = np.random.normal(size=R_test.shape, scale = 0.1)
R_test = R_test + noise
sns.scatterplot(x=R_test[:,0], y=R_test[:,1], hue = c_test)

Compare the training and test set. The latter is visibly noisier.

Now lets load the class `KNeighborsClassifier`. The second and third lines will instantiate an object that will perforn *k*-NN by looking at the 5 nearest neighbors of every point and train a model on the test set `R_train` with ground truth labels `c_train`. The last line will generate the label predictions for the test data based on the model generated from the training set and assign the outpout to `c_pred_knn`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(R_train, c_train)
c_pred_knn = neigh.predict(R_test)

Let's plot the classifier's predictions.

In [ ]:
sns.scatterplot(x=R_test[:,0], y=R_test[:,1], hue = c_pred_knn)
print(np.sum(c_test == c_pred_knn)/len(c_test))

The predictions were >90% correct and the the output looks quite similar to what we would expect from *k*-means.

Let's do something more interesting. We will modify the data generation parameters to make our data more complex and high dimensional. This data will have 5 features and 7 clusters. We will continue plotting just the first 2 features for visualization purposes.

In [ ]:
n_samples = 500
centers = 7
seed = 2
n_features = 5

Now let's generate the new training set.

In [ ]:
R_train, c_train = make_blobs(n_samples=n_samples, centers=centers, n_features=n_features, random_state=seed,  cluster_std=0.7)
sns.scatterplot(x=R_train[:,0], y=R_train[:,1], hue = c_train)

Now for the new test set. We're going to double the variance of both the clusters and the added noise.

In [ ]:
R_test, c_test = make_blobs(n_samples=n_samples, centers=centers, n_features=n_features, random_state=seed,  cluster_std=2.0)
noise = np.random.normal(size=R_test.shape, scale = 0.2)
R_test = R_test + noise
sns.scatterplot(x=R_test[:,0], y=R_test[:,1], hue = c_test)

This data is visibly noisier, and mind you we're only looking at two of the five features.

Now let's train the *k*-NN classifier on this data exactly the same way as before and plot the predictions.

In [ ]:
neigh2 = KNeighborsClassifier(n_neighbors=5)
neigh2.fit(R_train, c_train)
c_pred_knn = neigh2.predict(R_test)
sns.scatterplot(x=R_test[:,0], y=R_test[:,1], hue = c_pred_knn)

Not terrible.

Let's evaluate the prediction accuracy.

In [ ]:
'k-NN accuracy: {:0.1f}%'.format(np.sum(c_test==c_pred_knn)/len(c_test)*100)

This is pretty good given how gross the data looks.

## Summary
Now that you have (or maybe already had) some degree of familiarity with the concepts of dimensionality reduction, clustering, classification, and supervised and unsuperivsed algorithms, you can hopefully make an informed decision about when it's appropriate to use each one.

While these actual algorithmic examples showcased here might seem like trivial and pedagological use cases, these algorithms are actually used with little to no modification in genomic data analysis, unlike the ones in the R section of the problem set. For example, *k*-NN, both as is and in other flavors, is used for cell type identification in single-cell omic data and works superbly well. So much so that our lab and others have published variations of it tailored for this purpose. On the other hand, *k*-means works very poorly because cell type identification is a classification, not a clustering, problem. However this hasn't stopped people from using it. This is why it's important to understand the differences between different algorithmic paradigms and in what contexts to apply them.

Now for the actual homework assignment...

## Part 5: I promise it's the last one

In this last problem, you will apply the concepts from these sections to classify cells as healthy or sick. Here we have provided a snRNA-seq dataset from a mouse model of Huntington's disease, the same one as from part 3. This dataset contains >1000 *Drd2*-expressing striosomal SPNs from four mice, roughly half of which are from 6 month old zQ175DN HD model mice and the other half from same-age C57BL/6 control litter mates. This is an interesting data set because despite this mouse model expressing mutant Huntingtin with 175 CAG repeats, it has a very mild motor phenotype. In contrast, the R6/2 model from part 2 had 160 repeats and at only 9 weeks of age exhibited a phenotype so severe, that they would not have survived more than a few more weeks. The biology underlying this difference is a very interesting topic of research and may be discussed in the neurodegeneration module of the course.  The transcriptional phenotype of the zQ175 is also fairly subtle, which adds to the  challange.

We briefly considered providing human data for this problem, but that would have been too difficult. As you can see from [this publication](https://www.cell.com/neuron/pdf/S0896-6273(20)30475-X.pdf), even we had difficulty telling them apart.

This problem will be graded primarily on how well you can justify your choice of classification approach moreso than the accuracy of the labels, provided that the labels are at least somehwat correct.

We provided you with the following data:
* `cts_class`: A sparse matrix containing the gene counts in feature-by-cell format. No preprocessing has been done other than reducing the number of genes to only those that are highly expressed.
* `gene_data`: A data frame containing the ENSEMBL ID, gene name, chromosome, and biotype of each feature.
* `df_label`: A data frame containing the labels, most of which are set to `NA`, sans 35 known labels which you can use as a starting point.

As stated above, the `label` column containes the true labels of 35 cells selected at random. You may use these to determine the remainder of the missing labels.



In the last cell at the bottom of this notebook, you are asked to explain your method and justify your approach.

You can use any method you wish to determine the phenotypes, not just the classification and clustering methods discussed here. You may also use any dimensionality reduction and normalization scheme you wish, if you think you need them.

Run this cell to load the problem data.

In [ ]:
cts_class = mmread("data_assignment1/p5_counts_zQ_sc_classification.mtx")
gene_data = pd.read_table("data_assignment1/p5_rowdata_zQ_sc_classification.tsv", sep="\t", header=0)
df_label = pd.read_table("data_assignment1/p5_labels_zQ_sc_classification.tsv", sep="\t", header=0)

Write your solution in the following cell. The output of your classifier should be a set of predictions that you will store in a new column named `label_pred` inside `df_label`. `df_label['idx']` contains the cell column indices, which should not be modified.
Once you have your solution, make sure to run the subsequent cell which will write your solution to a TSV file and submit it with your notebook files.

In [ ]:
# Write your code here






df_label['label_pred'] = ... # Fill this in


Plot your result:

In [ ]:
# Fill this in

Save your result by running this cell. Feel free to change the file location, but keep the file name the same.

In [ ]:
df_label.to_csv('p5_predicted_labels.tsv', sep='\t', na_rep='NA', header=True, index=False)

Explain and justify your solution:

**Remember to save and download the notebook, as well as your solution to part 5.**